# Variação Paramétrica do SVM


# Passo 1: Preparando o ambiente

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report

In [2]:
DATASET_ORIGINAL = "../datasets/data_normalizada.csv"
DATASET_SMOTE = "../datasets/data_smote.csv"
DATASET_UNDER = "../datasets/data_under.csv"
DATASET_TESTE = "../datasets/data_test.csv"

In [3]:
df: pd.DataFrame = pd.read_csv(DATASET_ORIGINAL)
df_smote: pd.DataFrame = pd.read_csv(DATASET_SMOTE)
df_under: pd.DataFrame = pd.read_csv(DATASET_UNDER)
df_test: pd.DataFrame = pd.read_csv(DATASET_TESTE)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6819 entries, 0 to 6818
Data columns (total 94 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   Bankrupt?                                                6819 non-null   int64  
 1   ROA(C) before interest and depreciation before interest  6819 non-null   float64
 2   ROA(A) before interest and % after tax                   6819 non-null   float64
 3   ROA(B) before interest and depreciation after tax        6819 non-null   float64
 4   Operating Gross Margin                                   6819 non-null   float64
 5   Realized Sales Gross Margin                              6819 non-null   float64
 6   Operating Profit Rate                                    6819 non-null   float64
 7   Pre-tax net Interest Rate                                6819 non-null   float64
 8   After-tax net Interest Rate 

# Passo 2: Definição do Campo de busca

Aqui é definido os parâmetros que serão testados pelo modelo. Segue abaixo uma breve explicação dos parâmetros testados:

 - C (float, default=1.0): Regularização, quanto maior, mais fraca é a força de regularização
 - kernel ("linear", "poly", "rbf", "sigmoid", "precomputed" ou função, default="rbf"): kernel usado no algorítmo
 - degree (int, default=3): apenas para o kernel = poly
 - gamma ("scale", "auto" ou float, default="scale"): coeficiente do kernel para "rbf", "poly" e "sigmoid".
 - coef0 (float, default=0.0): Termo independente na função do kernel, apenas para os kernels "poly" e "sigmoid"
 - tol (float, default=1e-3): tolerância para a parada

Como há parâmetros que apenas funcionam para Kernels espeíficos, vamos separar os testes por Kernel pois o algorítmo de escolha de parâmetros será o GridSearchCV, que olha para todas as possibilidades e não faz sentido testar um modelo SVM com kernel linear com diferentes valores de degree, por exemplo.

In [5]:
svm_linear = {
    "C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "kernel": ["linear"],
    "tol": [1e-2, 1e-3, 1e-4]
}

In [6]:
svm_poly = {
    "C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "kernel": ["poly"],
    "degree": [2, 3, 4, 5],
    "gamma": ["scale", "auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0],
    "tol": [1e-2, 1e-3, 1e-4]
}

In [7]:
svm_rbf = {
    "C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "kernel": ["rbf"],
    "gamma": ["scale", "auto"],
    "tol": [1e-2, 1e-3, 1e-4]
}

In [8]:
svm_sigmoid = {
    "C": [0.1, 0.5, 1.0, 1.5, 2.0],
    "kernel": ["sigmoid"],
    "gamma": ["scale", "auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0],
    "tol": [1e-2, 1e-3, 1e-4]
}

# Passo 3: Definindo a Busca por Parâmetros

Nesse passo é usado o algorítmo GridSearchCV para escolher os melhores parâmetros, já que ele testa todas as possibilidades.

Para ele, foi definido:

n_jobs=-1: para utilizarmos todos os processadores possíveis e otimizar a execução;
cv=5: para fazer um KFold com 5 folds, dividindo o dataset em 5 partes;
scoring="f1": para escolher os melhores parâmetros a partir do f1, por ser um dataset desbalanceado;
error_score='raise': para propagar possíveis erros no treinamento com dado conjunto de parâmentros e parar o teste

Comparações de performance serão feitas nas próximas etapas.

In [9]:
def SVMSearch(search_params: list[list], X_train: np.ndarray, X_test:  np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> list:
    bests = []
    for search_param in search_params:
        svm = SVC()
        
        print(f"Buscando com os parametros {search_param}")
        
        svm_grid_searcher = GridSearchCV(svm, search_param, n_jobs=-1, error_score='raise', cv=5, scoring="f1")
        svm_best = svm_grid_searcher.fit(X_train, y_train)
        
        print(f"Melhores parâmetros: {svm_best.best_params_}")
        print(f"Melhor f1: {svm_best.best_score_}")
        
        print(classification_report(y_test, svm_best.predict(X_test)))
        bests.append(svm_best)
    
    return bests

# Passo 4: Realizando a busca para o dataset normalizado

In [10]:
X, y = df.iloc[:, 1:], df.iloc[:, 0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [11]:
bests = SVMSearch(
    [
        svm_linear,
        svm_poly,
        svm_rbf,
        svm_sigmoid
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'kernel': ['linear'], 'tol': [0.01, 0.001, 0.0001]}
Melhores parâmetros: {'C': 2.0, 'kernel': 'linear', 'tol': 0.01}
Melhor f1: 0.11600052246603969
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2637
           1       0.53      0.09      0.15        91

    accuracy                           0.97      2728
   macro avg       0.75      0.54      0.57      2728
weighted avg       0.95      0.97      0.96      2728

Buscando com os parametros {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'kernel': ['poly'], 'degree': [2, 3, 4, 5], 'gamma': ['scale', 'auto'], 'coef0': [0.0, 0.1, 0.5, 1.0], 'tol': [0.01, 0.001, 0.0001]}
Melhores parâmetros: {'C': 1.5, 'coef0': 1.0, 'degree': 5, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}
Melhor f1: 0.27073365231259966
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      2637
           1       0.33  

Podemos notar que os kernels "linear" e "rbf" não funcionaram bem para o nosso problema. Logo, vamos continuar a busca a partir dos outros kernels. Gamma="auto" sempre foi melhor, então vamos fixar ele.

In [12]:
svm_poly_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["poly"],
    "degree": [3, 4, 5, 6, 7, 8],
    "gamma": ["auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [13]:
svm_sigmoid_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["sigmoid"],
    "gamma": ["auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [14]:
bests = SVMSearch(
    [
        svm_poly_,
        svm_sigmoid_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'C': [1.5, 2.0, 2.5, 3, 3.5], 'kernel': ['poly'], 'degree': [3, 4, 5, 6, 7, 8], 'gamma': ['auto'], 'coef0': [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5], 'tol': [0.01, 0.001, 0.0001, 1e-05]}
Melhores parâmetros: {'C': 2.5, 'coef0': 1.5, 'degree': 6, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}
Melhor f1: 0.2980888843095839
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      2637
           1       0.31      0.35      0.33        91

    accuracy                           0.95      2728
   macro avg       0.64      0.66      0.65      2728
weighted avg       0.96      0.95      0.95      2728

Buscando com os parametros {'C': [1.5, 2.0, 2.5, 3, 3.5], 'kernel': ['sigmoid'], 'gamma': ['auto'], 'coef0': [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5], 'tol': [0.01, 0.001, 0.0001, 1e-05]}
Melhores parâmetros: {'C': 2.5, 'coef0': 0.5, 'gamma': 'auto', 'kernel': 'sigmoid', 'tol': 0.01}
Melhor f1: 0.29249924626462775
              p

Os melhores parâmetros foram: {'C': 2.5, 'coef0': 1.5, 'degree': 6, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}, com f1 macro = 0.65 

# Passo 5: Realizando a busca para o dataset smote

In [17]:
X_train, y_train = df_smote.iloc[:, 1:], df_smote.iloc[:, 0]
X_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

In [18]:
bests = SVMSearch(
    [
        svm_linear,
        svm_poly,
        svm_rbf,
        svm_sigmoid
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'kernel': ['linear'], 'tol': [0.01, 0.001, 0.0001]}
Melhores parâmetros: {'C': 1.5, 'kernel': 'linear', 'tol': 0.01}
Melhor f1: 0.9061849629229503
              precision    recall  f1-score   support

           0       0.99      0.88      0.93      2637
           1       0.19      0.79      0.30        91

    accuracy                           0.88      2728
   macro avg       0.59      0.84      0.62      2728
weighted avg       0.96      0.88      0.91      2728

Buscando com os parametros {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'kernel': ['poly'], 'degree': [2, 3, 4, 5], 'gamma': ['scale', 'auto'], 'coef0': [0.0, 0.1, 0.5, 1.0], 'tol': [0.01, 0.001, 0.0001]}
Melhores parâmetros: {'C': 2.0, 'coef0': 1.0, 'degree': 5, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}
Melhor f1: 0.9783742489005359
              precision    recall  f1-score   support

           0       0.98      0.95      0.97      2637
           1       0.22    

Os melhores resultados foram com os kernels "poly" e "rbf". Para ambos o melhor gamma foi "auto", então será fixado nos próximos testes.

In [19]:
svm_poly_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["poly"],
    "degree": [3, 4, 5, 6, 7, 8],
    "gamma": ["auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [20]:
svm_rbf_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["rbf"],
    "gamma": ["auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [21]:
bests = SVMSearch(
    [
        svm_poly_,
        svm_rbf_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'C': [1.5, 2.0, 2.5, 3, 3.5], 'kernel': ['poly'], 'degree': [3, 4, 5, 6, 7, 8], 'gamma': ['auto'], 'coef0': [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5], 'tol': [0.01, 0.001, 0.0001, 1e-05]}
Melhores parâmetros: {'C': 3, 'coef0': 2.5, 'degree': 4, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}
Melhor f1: 0.9820185584394624
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2637
           1       0.29      0.41      0.34        91

    accuracy                           0.95      2728
   macro avg       0.64      0.69      0.66      2728
weighted avg       0.96      0.95      0.95      2728

Buscando com os parametros {'C': [1.5, 2.0, 2.5, 3, 3.5], 'kernel': ['rbf'], 'gamma': ['auto'], 'coef0': [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5], 'tol': [0.01, 0.001, 0.0001, 1e-05]}
Melhores parâmetros: {'C': 3.5, 'coef0': 0.0, 'gamma': 'auto', 'kernel': 'rbf', 'tol': 0.01}
Melhor f1: 0.9680507177676463
              precision   

Analisando o f1 do teste (macro = 0.66), o modelo com os parâmetros {'C': 3, 'coef0': 2.5, 'degree': 4, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01} foi melhor

# Passo 6: Realizando a busca para o dataset under

In [22]:
X_train, y_train = df_under.iloc[:, 1:], df_under.iloc[:, 0]
X_test, y_test = df_test.iloc[:, 1:], df_test.iloc[:, 0]

In [23]:
bests = SVMSearch(
    [
        svm_linear,
        svm_poly,
        svm_rbf,
        svm_sigmoid
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'kernel': ['linear'], 'tol': [0.01, 0.001, 0.0001]}
Melhores parâmetros: {'C': 0.1, 'kernel': 'linear', 'tol': 0.01}
Melhor f1: 0.8188535414165667
              precision    recall  f1-score   support

           0       0.99      0.82      0.90      2637
           1       0.14      0.85      0.24        91

    accuracy                           0.82      2728
   macro avg       0.57      0.84      0.57      2728
weighted avg       0.97      0.82      0.88      2728

Buscando com os parametros {'C': [0.1, 0.5, 1.0, 1.5, 2.0], 'kernel': ['poly'], 'degree': [2, 3, 4, 5], 'gamma': ['scale', 'auto'], 'coef0': [0.0, 0.1, 0.5, 1.0], 'tol': [0.01, 0.001, 0.0001]}
Melhores parâmetros: {'C': 0.1, 'coef0': 1.0, 'degree': 2, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}
Melhor f1: 0.8475882352941178
              precision    recall  f1-score   support

           0       1.00      0.85      0.92      2637
           1       0.17    

Os melhores resultados foram com os kernels "poly", "sigmoid" e "rbf". Para ambos o melhor gamma foi "auto", então será fixado nos próximos testes.

In [24]:
svm_poly_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["poly"],
    "degree": [3, 4, 5, 6, 7, 8],
    "gamma": ["auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [25]:
svm_rbf_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["rbf"],
    "gamma": ["auto"],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [26]:
svm_sigmoid_ = {
    "C": [1.5, 2.0, 2.5, 3, 3.5],
    "kernel": ["sigmoid"],
    "gamma": ["auto"],
    "coef0": [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5],
    "tol": [1e-2, 1e-3, 1e-4, 1e-5]
}

In [27]:
bests = SVMSearch(
    [
        svm_poly_,
        svm_rbf_,
        svm_sigmoid_
    ],
    X_train, X_test, y_train, y_test
)

Buscando com os parametros {'C': [1.5, 2.0, 2.5, 3, 3.5], 'kernel': ['poly'], 'degree': [3, 4, 5, 6, 7, 8], 'gamma': ['auto'], 'coef0': [0.0, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5], 'tol': [0.01, 0.001, 0.0001, 1e-05]}
Melhores parâmetros: {'C': 1.5, 'coef0': 1.5, 'degree': 4, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01}
Melhor f1: 0.8358733031674209
              precision    recall  f1-score   support

           0       0.99      0.84      0.91      2637
           1       0.14      0.78      0.24        91

    accuracy                           0.83      2728
   macro avg       0.57      0.81      0.57      2728
weighted avg       0.96      0.83      0.88      2728

Buscando com os parametros {'C': [1.5, 2.0, 2.5, 3, 3.5], 'kernel': ['rbf'], 'gamma': ['auto'], 'tol': [0.01, 0.001, 0.0001, 1e-05]}
Melhores parâmetros: {'C': 2.0, 'gamma': 'auto', 'kernel': 'rbf', 'tol': 0.01}
Melhor f1: 0.8453921771901793
              precision    recall  f1-score   support

           0       1.00      

O melhor modelo foi com os parâmetros {'C': 1.5, 'coef0': 1.5, 'degree': 4, 'gamma': 'auto', 'kernel': 'poly', 'tol': 0.01} com f1 (macro) = 0.57